In [1]:
import os
import sys
import gensim
import pandas as pd
import pickle
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\User\Anaconda3\lib\site-packages\pattern-2.6-py3.6.egg\pattern\text\en\..\..\..\..\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using 

In [2]:
# train_set = pd.read_csv('./corpus/tripadvisor/train_set.csv')
test_set = pd.read_csv('./corpus/tripadvisor/test.csv')
test_set.head()

,content,polarity
0,lokasi hotel tidak jauh dari komplek mall kali...,negative
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative
2,lokasi hotel yang sangat strategis masih di ko...,negative
3,lokasi hotel yang sangat strategis masih di ko...,negative
4,lokasi hotel yang strategis di tengah kota dan...,negative


In [3]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [4]:
# train_set['sentiment'] = train_set['polarity'].apply(sentiment_label)
test_set['sentiment'] = test_set['polarity'].apply(sentiment_label)
test_set.head()

,content,polarity,sentiment
0,lokasi hotel tidak jauh dari komplek mall kali...,negative,0
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative,0
2,lokasi hotel yang sangat strategis masih di ko...,negative,0
3,lokasi hotel yang sangat strategis masih di ko...,negative,0
4,lokasi hotel yang strategis di tengah kota dan...,negative,0


In [5]:
SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(test_set['content'], test_set['sentiment'], test_size=.1, random_state=SEED)

In [6]:
# x_train = train_set['content']
# x_validation = test_set['content']
# y_train = train_set['sentiment']
# y_validation = test_set['sentiment']

In [7]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [8]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 700

In [9]:
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [10]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
746it [00:03, 199.15it/s]
83it [00:00, 209.16it/s]


In [11]:
batch_size = 56
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [12]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [13]:
model = Sequential()
model.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, y_train, epochs=num_epochs, validation_data=[val_vecs, y_validation])

Train on 746 samples, validate on 83 samples
Epoch 1/10
746/746 [==============================] - ETA: 1:41 - loss: 0.7662 - acc: 0.375 - ETA: 51s - loss: 0.7362 - acc: 0.437 - ETA: 34s - loss: 0.7116 - acc: 0.50 - ETA: 25s - loss: 0.7143 - acc: 0.50 - ETA: 20s - loss: 0.7017 - acc: 0.52 - ETA: 16s - loss: 0.6966 - acc: 0.52 - ETA: 13s - loss: 0.6855 - acc: 0.54 - ETA: 11s - loss: 0.6753 - acc: 0.55 - ETA: 10s - loss: 0.6713 - acc: 0.55 - ETA: 8s - loss: 0.6641 - acc: 0.5781 - ETA: 7s - loss: 0.6529 - acc: 0.605 - ETA: 6s - loss: 0.6438 - acc: 0.617 - ETA: 5s - loss: 0.6410 - acc: 0.625 - ETA: 5s - loss: 0.6402 - acc: 0.629 - ETA: 4s - loss: 0.6318 - acc: 0.639 - ETA: 3s - loss: 0.6262 - acc: 0.644 - ETA: 3s - loss: 0.6189 - acc: 0.648 - ETA: 2s - loss: 0.6118 - acc: 0.659 - ETA: 2s - loss: 0.6090 - acc: 0.666 - ETA: 1s - loss: 0.6028 - acc: 0.675 - ETA: 1s - loss: 0.6009 - acc: 0.678 - ETA: 0s - loss: 0.5993 - acc: 0.677 - ETA: 0s - loss: 0.5952 - acc: 0.679 - 11s 14ms/step - loss: 0

746/746 [==============================] - ETA: 4s - loss: 0.0296 - acc: 1.000 - ETA: 4s - loss: 0.0277 - acc: 1.000 - ETA: 4s - loss: 0.0308 - acc: 1.000 - ETA: 4s - loss: 0.0286 - acc: 1.000 - ETA: 4s - loss: 0.0268 - acc: 1.000 - ETA: 3s - loss: 0.0301 - acc: 1.000 - ETA: 3s - loss: 0.0281 - acc: 1.000 - ETA: 3s - loss: 0.0280 - acc: 1.000 - ETA: 3s - loss: 0.0280 - acc: 1.000 - ETA: 3s - loss: 0.0284 - acc: 1.000 - ETA: 2s - loss: 0.0351 - acc: 0.994 - ETA: 2s - loss: 0.0340 - acc: 0.994 - ETA: 2s - loss: 0.0335 - acc: 0.995 - ETA: 2s - loss: 0.0342 - acc: 0.995 - ETA: 1s - loss: 0.0337 - acc: 0.995 - ETA: 1s - loss: 0.0326 - acc: 0.996 - ETA: 1s - loss: 0.0314 - acc: 0.996 - ETA: 1s - loss: 0.0316 - acc: 0.996 - ETA: 0s - loss: 0.0317 - acc: 0.996 - ETA: 0s - loss: 0.0311 - acc: 0.996 - ETA: 0s - loss: 0.0302 - acc: 0.997 - ETA: 0s - loss: 0.0313 - acc: 0.995 - ETA: 0s - loss: 0.0314 - acc: 0.995 - 6s 8ms/step - loss: 0.0311 - acc: 0.9960 - val_loss: 0.1014 - val_acc: 0.9639
Epoch

In [14]:
# model.save('./model/bi_lstm_pv/bi_lstm_model_01.h5')  

In [15]:
# model = load_model('./model/bi_lstm_pv/bi_lstm_model_01.h5')
y_pred = model.predict(val_vecs)
for i in range(len(y_pred)):
    y_pred[i][0] = round(y_pred[i][0])

print("Accuracy: ", accuracy_score(y_validation, y_pred))
print(classification_report(y_validation, y_pred, labels = [0, 1], digits=8))

Accuracy:  0.9759036144578314
             precision    recall  f1-score   support

          0  0.96491228 1.00000000 0.98214286        55
          1  1.00000000 0.92857143 0.96296296        28

avg / total  0.97674910 0.97590361 0.97567253        83

